In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random

import os
import cv2
import glob
import shutil
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [2]:
random.seed(0)
np.random.seed(0)

In [3]:
WORKING = "/kaggle/temp/"

In [4]:
os.makedirs(WORKING+"train", exist_ok=True)
os.makedirs(WORKING+"val", exist_ok=True)
os.makedirs(WORKING+"test", exist_ok=True)

for i in ["train", "val", "test"]:
    for j in ["images", "labels"]:
        os.makedirs(WORKING + i +"/"+j, exist_ok=True)

In [5]:
img_files = glob.glob("/kaggle/input/bird-dataset/*/*/*.jpg")
random.shuffle(img_files)

In [6]:
raw_labels = {}
bird_images = []
for i in ["1", "2"]:
    with open("/kaggle/input/bird-dataset/BIRD_v210_2.txt") as f:
        raw = f.read().strip()
        last_name = None
        for line in raw.split():
            if line.endswith(".jpg"):
                last_name = line
                raw_labels[last_name] = []
                bird_images.append(line)
            splitted = line.split(",")
            if splitted[-1] == "b":
                raw_labels[last_name].append(["0"] + splitted[:-1]) # set category to 0 since only 1 object to detect

In [7]:
stratify_label = {}
for x in img_files:
    label = x.split("/")[4].split("_")[2]
    label += "1" if (x.split("/")[-1] in bird_images) else "0"
    stratify_label[x] = label

In [8]:
train_test_x, val_x = train_test_split(img_files, train_size=0.9, stratify=list(stratify_label.values()), random_state=0)
new_stratify = [stratify_label[x] for x in train_test_x]
train_x, test_x = train_test_split(train_test_x, train_size=(0.8)/(0.9), stratify=new_stratify, random_state=0)

In [9]:
for x in train_x:
    with open(WORKING+"train/labels/"+x.split("/")[-1].split(".")[0]+".txt", "w") as file:
        if x.split("/")[-1] in raw_labels:
            lines = [" ".join(y) for y in raw_labels[x.split("/")[-1]]]
            file.write("\n".join(lines))

In [10]:
for x in tqdm(train_x):
    shutil.copy2(x, WORKING+"train/images")
for x in tqdm(val_x):
    shutil.copy2(x, WORKING+"val/images")
for x in tqdm(test_x):
    shutil.copy2(x, WORKING+"test/images")

100%|██████████| 1080/1080 [00:54<00:00, 19.74it/s]


In [11]:
!cd /kaggle/working
!git clone https://github.com/WongKinYiu/yolov7.git

Cloning into 'yolov7'...
remote: Enumerating objects: 1127, done.
remote: Total 1127 (delta 0), reused 0 (delta 0), pack-reused 1127
Receiving objects: 100% (1127/1127), 69.93 MiB | 1.38 MiB/s, done.
Resolving deltas: 100% (525/525), done.


In [12]:
!pip install -r /kaggle/working/yolov7/requirements.txt
!sudo apt install -y zip htop screen libgl1-mesa-glx




htop is already the newest version (2.2.0-2build1).
zip is already the newest version (3.0-11build1).
libgl1-mesa-glx is already the newest version (21.2.6-0ubuntu0.1~20.04.2).
screen is already the newest version (4.8.0-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 93 not upgraded.


In [13]:
f = open("/kaggle/working/yolov7/data/custom.yaml", "w")
f.write("""train: /kaggle/temp/train/
test: /kaggle/temp/test/
val: /kaggle/temp/val/

nc: 1

names: ['bird']
""")
f.close()

In [14]:
os.chdir("/kaggle/working/yolov7")
!python detect.py --weights yolov7.pt --source inference/images/horses.jpg --img 640

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=False, img_size=640, iou_thres=0.45, name='exp', no_trace=False, nosave=False, project='runs/detect', save_conf=False, save_txt=False, source='inference/images/horses.jpg', update=False, view_img=False, weights=['yolov7.pt'])
100%|██████████████████████████████████████| 72.1M/72.1M [00:06<00:00, 11.6MB/s]

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

/opt/conda/lib/python3.7/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /usr/local/src/pytorch/aten/src/ATen/native/TensorShape.cpp:2227.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
5 horses, Done. (22.0ms) Inference, (24.3ms) NMS
 The image with the result is save

In [15]:
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

wandb.login(key=user_secrets.get_secret("wandb"))
wandb.init(project="yolov7-bird",name=f"run3")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: unkownhihi. Use `wandb login --relogin` to force relogin


In [16]:
!python3 train.py --batch 64  --epochs 50 --weights yolov7.pt --data "/kaggle/working/yolov7/data/custom.yaml" --cfg cfg/training/yolov7.yaml --name 'run3' --project 'yolov7-bird'

wandb: Currently logged in as: unkownhihi. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.13.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.12.21
wandb: Run data is saved locally in /kaggle/working/yolov7/wandb/run-20230109_224125-13xpm1n6
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run run3
wandb: ⭐️ View project at https://wandb.ai/unkownhihi/yolov7-bird
wandb: 🚀 View run at https://wandb.ai/unkownhihi/yolov7-bird/runs/13xpm1n6







































































































































































































































































































































































































































































In [17]:
!cp /kaggle/temp/yolov7/runs/train/yolov7-bird/weights/best.pt /kaggle/working/best.pt

cp: cannot stat '/kaggle/temp/yolov7/runs/train/yolov7-bird/weights/best.pt': No such file or directory
